In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Custom Transactions Using Custom Properties

This notebook shows how to create custom transaction types that pass reference values (units, instruments, exchange rates) from custom transaction properties into the Movements Engine.

Attributes
----------
transactions types
sides
properties
portfolio
holdings
FX
"""

toggle_code("Toggle Docstring")

# Custom Transactions Using Custom Properties

This notebook shows how to create custom transaction types that pass reference values (units, instruments, exchange rates) from custom transaction properties into the Movements Engine. This means that it is possible to upsert transactions that do not use the standard transaction fields as points of reference, as such, providing a solution for bespoke transactions fields that better represent a user's interpretation of some transaction types, e.g. FX transactions.

## The Challenge

For the following data, normal usage would require each field to be mapped to a value on our standard model. In this case, the `BuyAmount` would map to the `Units` field, the `BuyAmountCcy` would map to the `InstrumentIdentifiers` and the `SellAmountCcy` would map to the `TotalConsideraton.Currency`.  This method works but these mappings are not always intuitive eg. when considering an FX swap.

<br/><br/>

| TransactionType | BuyAmount | SellAmount | ExchangeRate | BuyAmountCcy | SellAmountCcy | TradeDate            | SettleDate           |
|-----------------|-----------|------------|--------------|--------------|---------------|----------------------|----------------------|
| FX              | 1000      | 800        | 0.8          | EUR          | GBP           | 2021-01-01T00:00:00Z | 2021-01-03T00:00:00Z |
| FX              | 400       | 508        | 1.27         | GBP          | EUR           | 2021-01-01T00:00:00Z | 2021-01-03T00:00:00Z |
| FX              | 1200      | 1000       | 1.2          | USD          | GBP           | 2021-01-01T00:00:00Z | 2021-01-03T00:00:00Z |



## The Solution

We will use the example of FX swaps, but by following the same method and changing the transaction properties, this notebook will be suitable for other transaction types.

We are able to leverage custom properties and custom side definitions to build a custom transaction type that fits the FX swaps use case. The flexibility that Lusid and the Movement Engine provides makes this process simple.

By the end of this notebook, you will have a portfolio that is able to generate holdings by utilising the properties of a transaction instead of using mappings.

The steps are as follows:
1. [Initialise Notebook](#1.-Initialise-Notebook)
2. [Create Transaction Properties](#2.-Create-Transaction-Properties)
3. [Create Custom Sides](#3.-Create-Custom-Sides)
4. [Create Custom Transaction Type](#4.-Create-Custom-Transaction-Type)
5. [Create Portfolio](#5.-Create-Portfolio)
6. [Upsert Transactions](#6.-Upsert-Transactions)
7. [Get Holdings](#7.-Get-Holdings)
8. [Extras](#8.-Extras)


## 1. Initialise Notebook
Import packages, validate client, load data and define scope.

In [2]:
# Import general purpose Python packages
import pandas as pd
from datetime import datetime
import pytz
from IPython.core.display import HTML

# Import LUSID specific packages
import lusid as lu
import lusid.models as models
from lusidjam import RefreshingToken

# Establish scope and portfolio details
scope = "load-transaction-demo"
code = "demo-portfolio"
name = "DemoPortfolio"
data_path = "./data/data.csv"

# Authenticate our user and create our API client
secrets_path=os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path=os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory=lu.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status=pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.11050.0,0.5.3206,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
# Initialise api
txn_config_api = api_factory.build(lu.api.TransactionConfigurationApi)
txn_portfolio_api = api_factory.build(lu.api.TransactionPortfoliosApi)
property_definition_api = api_factory.build(lu.api.PropertyDefinitionsApi)

In [4]:
# Get data
data = pd.read_csv(data_path)
data

,TransactionType,BuyAmount,SellAmount,ExchangeRate,BuyAmountCcy,SellAmountCcy,TradeDate,SettleDate
0,FX,1000,800,0.80,EUR,GBP,2021-01-01T00:00:00Z,2021-01-03T00:00:00Z
1,FX,400,508,1.27,GBP,EUR,2021-01-01T00:00:00Z,2021-01-03T00:00:00Z
2,FX,1200,1000,1.20,USD,GBP,2021-01-01T00:00:00Z,2021-01-03T00:00:00Z


## 2. Create Transaction Properties

First, we need to create a property for each of our column values.

Note that for now, we require a separate security property due to the difference in the way securities and currency fields process currencies. Soon we will be able to use the same property for both.

In [5]:
# We only need these columns from the dataframe
columns = ["BuyAmount", "SellAmount", "ExchangeRate", "BuyAmountCcy", "SellAmountCcy", "BuyAmountCcySecurity", "SellAmountCcySecurity"]

for prop in columns:
    # These values are numerical, the rest are strings
    data_type = "number" if prop in ["BuyAmount", "SellAmount", "ExchangeRate"] else "string"
    # Define the property
    body = models.CreatePropertyDefinitionRequest(
        domain="Transaction",
        scope=scope,
        code=prop,
        display_name=prop,
        data_type_id = models.ResourceId(
            scope="system",
            code=data_type
        )
    )
    # Create the property
    try:
        response = property_definition_api.create_property_definition(create_property_definition_request=body)
        print(f"Property '{response.code}' created")
        
    except lu.ApiException as e:
        if json.loads(e.body)["code"] == 124: # PropertyAlreadyExists
            print(json.loads(e.body)["title"])
        else:
            raise e

Property 'BuyAmount' created
Property 'SellAmount' created
Property 'ExchangeRate' created
Property 'BuyAmountCcy' created
Property 'SellAmountCcy' created
Property 'BuyAmountCcySecurity' created
Property 'SellAmountCcySecurity' created


## 3. Create Custom Sides

We need to create two sides, one for moving the currency out of our portfolio (sell) and the other to bring currency in (buy). You can conceptualise sides as a collection of pointers which show the location of our properties. When we run our transaction, the movements engine looks at the location specified in the side and pulls the values down from there for each field. The values held at those locations are specified each time we create a new transaction (see [upsert transactions](#upsert_transactions)).

Our sides take the following fields and values:
<br></br>

| Field      | Buy Side                 | Sell Side        | Description                        |
|------------|--------------------------|------------------|------------------------------------|
| `side`     | BuyCurrencySide          | SellCurrencySide | The unique name for the side.      |
| `security` | BuyAmountCcy             | SellAmountCcy    | The security to use for the side.  |
| `currency` | BuyAmountCcy             | SellAmountCcy    | The currency to use for the side.  |
| `rate`     | Txn:TradeToPortfolioRate | ExchangeRate     | The rate to use for the side.      |
| `units`    | BuyAmount                | SellAmount       | The units to use for the side.     |
| `amount`   | BuyAmount                | SellAmount       | The amount to use for the side.    |

See our [support docs](https://support.lusid.com/knowledgebase/article/KA-01875/en-us) for more details on creating custom sides.

In [6]:
# Create request using properties declared above
buy_side_definition_request = models.SideDefinitionRequest(
    security="Transaction/load-transaction-demo/BuyAmountCcySecurity", # what currency are we buying?
    currency="Transaction/load-transaction-demo/BuyAmountCcy", # currency of security
    rate="Txn:TradeToPortfolioRate", # rate to portfolio currency
    units="Transaction/load-transaction-demo/BuyAmount", # eg. £10 = 10 units
    amount="Transaction/load-transaction-demo/BuyAmount" # eg. £10
)

sell_side_definition_request = models.SideDefinitionRequest(
    security="Transaction/load-transaction-demo/SellAmountCcySecurity", # what currency are we selling?
    currency="Transaction/load-transaction-demo/SellAmountCcy", # currency of security
    rate="Transaction/load-transaction-demo/ExchangeRate", # quoted rate for FX swap
    units="Transaction/load-transaction-demo/SellAmount", # eg. $8 = 8 units
    amount="Transaction/load-transaction-demo/SellAmount" # eg. $8
)

# Declare the sides
try:
    buy_response = txn_config_api.set_side_definition(
        side="BuyCurrencySide",
        side_definition_request=buy_side_definition_request
    )
    print(f"Side '{buy_response.side}' created")
    
except lu.ApiException as e:
    raise e

try:
    sell_response = txn_config_api.set_side_definition(
        side="SellCurrencySide",
        side_definition_request=sell_side_definition_request
    )
    print(f"Side '{sell_response.side}' created")

except lu.ApiExceptions as e:
    raise e

Side 'BuyCurrencySide' created
Side 'SellCurrencySide' created


## 4. Create Custom Transaction Type

The custom transaction type is the central component of any transaction, and its flexibility is what allows us to be able to model these FX swaps.

Each transaction type contains two objects. Firstly, the aliases. Aliases in this context specify different identities that a transaction type can have. In this example we will only use a single alias, but in theory we could create as many as we'd like. This is particularly useful in a system where multiple terms our used to represent the same action eg. BUY and BY.
<br></br>

| Fields            | Definition                          | Value          |
|-------------------|-------------------------------------|----------------|
| type              | Name of the transaction type        | FX             |
| description       | Description of the transaction type | FX Transaction |
| transaction class | Transaction type grouping system    | Basic          |
| transaction roles | Transaction type grouping system    | Longer         |

The second essential component of transaction types are movements. A transaction type can have one or more movements, the role of which is to instruct the movements engine on how to deal with portfolio funds during a transaction. In our case, we use 2 movements with the first representing the buy action and the second for the sell action. The allocation and movement of funds is determined by the direction of the movement with a 1 denoting an increase and -1 a decrease. A movement is made up of the following components.
<br></br>

| Fields            | Definition                                                                 | Buy Value       | Sell Value       |
|-------------------|----------------------------------------------------------------------------|-----------------|------------------|
| movement type     | How and when a holding is updated                                          | CashReceivable  | CashCommitment   |
| side              | Determines which economic attributes of the transaction impact the holding | BuyCurrencySide | SellCurrencySide |
| direction         | Determines if change is an increase or decrease                            | 1               | -1               |

The transaction must also be assigned a `source`, a grouping mechanism, and a `type`. The `type` is the primary alias that we will use to uniquely identify this transaction type eg. 'FX'.

See our [support docs](https://support.lusid.com/knowledgebase/article/KA-01872/) for more details on creating custom transaction types.

In [7]:
transaction_type_request = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type="FX", # Label
            description="FX Transaction",
            transaction_class="Basic",
            transaction_roles="Longer",
        )
    ], 
    movements=[
        # Buy side movement
        models.TransactionTypeMovement(
            movement_types="CashReceivable", 
            side="BuyCurrencySide", # Buy properties
            direction=1, # Increase
        ),
        # Sell side movement
        models.TransactionTypeMovement(
            movement_types="CashCommitment",
            side="SellCurrencySide", # Sell properties
            direction=-1, # Decrease
        )
    ],
)

try:
    response = txn_config_api.set_transaction_type(
        source="default",
        type="FX",
        transaction_type_request=transaction_type_request
    )
    print(f"Type '{response.aliases[0].type}' created")
    
except lu.ApiException as e:
    raise e

Type 'FX' created


## 5. Create Portfolio

We need to create a portfolio to store our transactions in.

It is worth noting that when creating a portfolio, the creation date must precede, or at least be equal to, the trade date of any transactions.

In [8]:
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=name,
    code=code,
    base_currency="USD",
    created=datetime(2020, 1, 1, tzinfo=pytz.utc)
)

try:
    response = txn_portfolio_api.create_portfolio(
        scope,
        create_transaction_portfolio_request=portfolio_request
    )
    print(f"Portfolio '{response.display_name}' created")
    
except lu.ApiException as e:
    if json.loads(e.body)["code"] == 112: # PortfolioWithIdAlreadyExists
        print(json.loads(e.body)["title"])
    else:
        raise e

Portfolio 'DemoPortfolio' created


## 6. Upsert Transactions

We can now upsert transactions using the values from our data file. Note that values are set to 0 for the usual transaction fields, and instead the data is added within the properties. This allows us to pass validation without impacting our holdings.

It is within the properties of our transaction that we are able to add our real values. We do so by providing a property value for the properties we instantiated earlier in this notebook.

In [9]:
transactions = []

# Make transactions from our data
for index, row in data.iterrows():
    txn = models.TransactionRequest(
        transaction_id=f"txn{index}",
        type=row["TransactionType"],
        instrument_identifiers={"Instrument/default/Currency": row["BuyAmountCcy"]},
        transaction_date=row["TradeDate"],
        settlement_date=row["SettleDate"],
        units=0, # Required to pass validation
        total_consideration=models.CurrencyAndAmount(amount=0, currency=row["BuyAmountCcy"]),
        properties={
            # Set the properties
            "Transaction/load-transaction-demo/BuyAmount": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/BuyAmount",
                value=models.PropertyValue(
                    metric_value=models.MetricValue(
                        value=float(row["BuyAmount"]) # Pull values straight from data
                    )
                )
            ),
            "Transaction/load-transaction-demo/SellAmount": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/SellAmount",
                value=models.PropertyValue(
                    metric_value=models.MetricValue(
                        value=float(row["SellAmount"])
                    )
                )
            ),
            "Transaction/load-transaction-demo/ExchangeRate": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/ExchangeRate",
                value=models.PropertyValue(
                    metric_value=models.MetricValue(
                        value=float(row["ExchangeRate"])
                    )
                )
            ),
            "Transaction/load-transaction-demo/BuyAmountCcy": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/BuyAmountCcy",
                value=models.PropertyValue(
                    label_value=row["BuyAmountCcy"]
                )
            ),
            "Transaction/load-transaction-demo/SellAmountCcy": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/SellAmountCcy",
                value=models.PropertyValue(
                    label_value=row["SellAmountCcy"]
                )
            ),
            "Transaction/load-transaction-demo/BuyAmountCcySecurity": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/BuyAmountCcySecurity",
                value=models.PropertyValue(
                    label_value=row["BuyAmountCcy"]
                )
            ),
            "Transaction/load-transaction-demo/SellAmountCcySecurity": models.PerpetualProperty(
                key="Transaction/load-transaction-demo/SellAmountCcySecurity",
                value=models.PropertyValue(
                    label_value=row["SellAmountCcy"]
                )
            )
        }
    )
    transactions.append(txn)

# Upsert transactions to our portfolio
try:
    response = txn_portfolio_api.upsert_transactions(
        scope=scope,
        code=code,
        transaction_request=transactions
    )
    print(f"Transactions successfully upserted at {response.version.as_at_date}")
except lu.ApiException as e:
    raise e

Transactions successfully upserted at 2023-03-29 09:03:12.698056+00:00


## 7. Get Holdings

Finally, we need to verify that our transactions appear as they should. This can be seen either via the UI or by calling the `get_holdings` api.

In [10]:
def get_holdings(scope, code):
    holdings_response = txn_portfolio_api.get_holdings(
        scope=scope,
        code=code,
    )

    data = holdings_response.values

    data_structure = {
        "Currency": [i.currency for i in data],
        "Units": [i.cost.amount for i in data],
        "SettledUnits": [i.settled_units for i in data],
        "HoldingType": [i.holding_type_name for i in data]
    }

    return pd.DataFrame(data=data_structure)

get_holdings(scope, code)

,Currency,Units,SettledUnits,HoldingType
0,EUR,492.0,492.0,Balance
1,GBP,-1400.0,-1400.0,Balance
2,USD,1200.0,1200.0,Balance


Or view in UI

In [11]:
api_url = api_factory.api_client.configuration._base_path.replace("api","")
display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope={scope}&code={code}&entityType=Portfolio" target="_blank">See holdings positions in LUSID</a>'))

## 8. Extras

Let's see how this model reacts under different conditions.

### Non-settled Transactions

Let's see how this model reacts to transactions that are unsettled.

First we upsert a transaction with a settle day far in the future, but a historical trade date.

In [12]:
transactions = []

txn = models.TransactionRequest(
    transaction_id="txn3",
    type="FX",
    instrument_identifiers={"Instrument/default/Currency": "USD"},
    transaction_date=datetime(2021, 1, 1, tzinfo=pytz.utc).isoformat(),
    # Settle far in the future
    settlement_date=datetime(2099, 1, 1, tzinfo=pytz.utc).isoformat(),
    units=0, # Required to pass validation
    total_consideration=models.CurrencyAndAmount(amount=0, currency="USD"),
    properties={
        # Populate values to properties
        "Transaction/load-transaction-demo/BuyAmount": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/BuyAmount",
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=600
                )
            )
        ),
        "Transaction/load-transaction-demo/SellAmount": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/SellAmount",
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=500
                )
            )
        ),
        "Transaction/load-transaction-demo/ExchangeRate": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/ExchangeRate",
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=1.2
                )
            )
        ),
        "Transaction/load-transaction-demo/BuyAmountCcy": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/BuyAmountCcy",
            value=models.PropertyValue(
                label_value="USD"
            )
        ),
        "Transaction/load-transaction-demo/SellAmountCcy": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/SellAmountCcy",
            value=models.PropertyValue(
                label_value="GBP"
            )
        ),
        "Transaction/load-transaction-demo/BuyAmountCcySecurity": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/BuyAmountCcySecurity",
            value=models.PropertyValue(
                label_value="CCY_USD"
            )
        ),
        "Transaction/load-transaction-demo/SellAmountCcySecurity": models.PerpetualProperty(
            key="Transaction/load-transaction-demo/SellAmountCcySecurity",
            value=models.PropertyValue(
                label_value="CCY_GBP"
            )
        )
    }
)

# Upsert transactions to our portfolio
try:
    response = txn_portfolio_api.upsert_transactions(
        scope=scope,
        code=code,
        transaction_request=[txn]
    )
    print(f"Transactions successfully upserted at {response.version.as_at_date}")
except lu.ApiException as e:
    raise e

Transactions successfully upserted at 2023-03-29 09:03:13.524778+00:00


Next, get holdings to verify unsettled currency swap exists.

In [13]:
get_holdings(scope, code)

,Currency,Units,SettledUnits,HoldingType
0,EUR,492.0,492.0,Balance
1,GBP,-1400.0,-1400.0,Balance
2,USD,1200.0,1200.0,Balance
3,USD,600.0,0.0,Receivable
4,GBP,-500.0,0.0,CashCommitment


See the additional CashCommitment and Receivable entries which show the movement of funds that has not yet completed.